# 24.3 Dijkstra' s algorithm
Dijkstras' algorithm tackles the single-source shortest-paths (SSSP) problem on a weighted, directed graph whose edges have **non-negative** weight. It is a **greedy algorithm**, in which:
* It maintains a set $S$ with vertices, whose SSSP have been determined
* It keeps adding vertices to $S$, whose distance to source vertice is the shortest
* When a new vertice is added to $S$, the edges leaving u are relaxed

## Correctness
Although it is a greedy algorithm, the correctness can be proved.

## Running time analysis
The running time of Dijstra's algorithm depends on how we maintain $S$.
#### If $S$ is a normal set:
The running time is $O(V^2+E)=O(V^2)$, mainly because has to extract the minimum from a set containing $|V|$ vertices, which take time $O(V)$ and do it for every vertice; relax takes O(1) time and we need to do it for all edges.
#### If $S$ is a binary heap
The time to extract minimum can be reduced to $\lg V$, while relax is increased to $\lg (V)$. Thus the total time is $O(V\lg V +E\lg V)$. As such, a binary heap is worthy when $|E|$ is small compared to $|V|^2$.
#### If $S$ is a Fobonacci heap
The running time is $O(V \lg V+E)$.

## Python codes
The following functions `dijkstra` and `dijkstra_heap` implement Dijkstra's algorithm when $S$ is a normal set and a binary heap respectively.

In [260]:
def initialize_single_source(G,s):
    # assign attribute d for each vertex for inf, except for s
    global D
    D={k:10000 for k in G}
    D[s]=0
    
    # assign attribute pi for each vertex
    # now none of the vertex has a parent
    global PI
    PI={}    
    return D, PI

def relax(u,v,w):
    # given that variable D and PI are defined
    if D[v]>D[u]+w[u][v]:
        D[v]=D[u]+w[u][v]
        PI[v]=u
def extract_min(S):
    """
    1) return a vertex from S, whose distance* is the smallest
    2) the vertex is poped from S
    It iterates S once
    *the distance of vertices are stored in D
    """
    min_d=100000
    for i in S:      
        if D[i]<min_d:
            min_d=D[i]
            v_min_d=i
    S.remove(v_min_d)
    return v_min_d

def dijkstra(G,w,s):
    initialize_single_source(G,s)
    S={}
    Q=set(G.keys())

    while len(Q)>0:
        u=extract_min(Q)
        S[u]=D[u]
        for v in G[u]:
            relax(u,v,w)
 
    return S, PI

It returns SSSP from source vertex $s$ and the predecessors of vertices in Figure 24.6:
<img src="img/fig24.6.png">

In [261]:
G={'s':['t','y'],
  't':['x','y'],
  'x':['z'],
  'y':['t','x','z'],
  'z':['x','s']}
w={'s':{'t':10,'y':5},
  't':{'x':1,'y':2},
  'x':{'z':4},
  'y':{'t':3,'x':9,'z':2},
  'z':{'x':6,'s':7}}
dijkstra(G,w,'s')

({'s': 0, 'y': 5, 'z': 7, 't': 8, 'x': 9},
 {'t': 'y', 'y': 's', 'x': 't', 'z': 'y'})

Below `dijkstra_heap` is the immplementation of the minimum-priority queue with a binary heap. Here everything is built from scratch followed pseudocodes in Chapter 4. Otherwise, `heapq` can be imported from `collections`.

In [286]:
def left (i):
    """given the index of a node in a binary heap, return the index of its left child"""
    return (i)*2+1
def right (i):
    """given the index of a node in a binary heap, return the index of its right child"""
    return (i)*2+2
def parent(i):
    """given the index of a node in a binary heap, return the index of its parent"""
    return int((i-1)/2)
class vertex:
    def __init__(self,key=None,d=None,parent=None):
        self.key=key
        self.d=d
        self.parent=None
    
def initialize_single_source_heap(G,s):
    S=[]
    for i in G:
        if i!=s:
            S.append(vertex(key=i, d=10000,parent=None))
        else:
            S.append(vertex(key=i, d=0, parent=None))
    S=build_min_heap(S)
    
    S={i.key: i for i in S}
    return S

def min_heapify (arr, i):
    """maintain min-heap property for a linear array arr at position i"""
    l = left (i)
    r = right (i)
    heap_size= len(arr)
    
    # compare key with its right child
    if l< heap_size and arr[l].d<arr[i].d:
        smallest=l    
    else:
        smallest=i
        
    # compare key with its left child
    if r<heap_size and arr[r].d<arr[smallest].d:
        smallest=r
        
    # if i is not the smallest, the the key should be swapped with the smallest
    # stop until i becomes the smallest
    if smallest != i:
        arr[i],arr[smallest]=arr[smallest],arr[i]
        min_heapify(arr,smallest)
        
    return arr

def node_start(a):
    """given the size of a linear array, return the index of the first node in a binary heap"""
    return int((a-2)/2)
def build_min_heap(arr):
    """given a linear array arr, build a minimum heap"""
    heap_size= len(arr)
    for n in range(node_start(heap_size),-1,-1):
        min_heapify(arr,n)
        
    return arr
def heap_extract_min(arr):
    """given a minimum heap, returns:
    1) the minimum (top node) of the heap
    2) a minimum heap formed by the rest of nodes
    """
    arr=list(arr.values())
    heap_size=len(arr)
    if heap_size>1:
            mini=arr[0]
            arr[0]=arr[-1]
            arr=min_heapify(arr[:-1],0)#+arr[]
            return mini, {i.key: i for i in arr}
    else:
        return arr[0], {}
def heap_decrease_key(arr,i,key):
    """given a minimum heap, the index of a node whose key we want to decrease,
    and the value of the new key,
    return the array with updated key of the node while maintaining min-heap property
    * notice that the new key must be smaller than the current one,
    otherwise the algo does not work (then we need to swap with childs, instead of parent)
    """
    
    if key>arr[i].d:
        pass
    else:
        i=arr[i]
        arr=list(arr.values())
        i=arr.index(i)

        # while the key of parent is larger than key
        # move upward
        while i>0 and arr[parent(i)].d>key:
            arr[i],arr[parent(i)]=arr[parent(i)],arr[i]
            i=parent(i)
        arr[i].d=key
    
    return {i.key: i for i in arr}
    
def dijkstra_heap(G,w,s):
    Q=initialize_single_source_heap(G,s)
    S=[]
     
    while len(Q)>0:
        u, Q= heap_extract_min(Q)
        S.append(u)
        
        for v in G[u.key]:
            try:
                if Q[v].d>u.d+w[u.key][v]:
                    Q[v].parent=u.key
                    Q=heap_decrease_key(Q,v,u.d+w[u.key][v] )
            except KeyError:
                pass
            
    return S

Running `dijkstra_heap` on Figure 24.6 gives the same results:

In [287]:
G={'s':['t','y'],
  't':['x','y'],
  'x':['z'],
  'y':['t','x','z'],
  'z':['x','s']}
w={'s':{'t':10,'y':5},
  't':{'x':1,'y':2},
  'x':{'z':4},
  'y':{'t':3,'x':9,'z':2},
  'z':{'x':6,'s':7}}
sssp=dijkstra_heap(G,w,'s')
print ([i.key for i in sssp])
print ([i.parent for i in sssp])
print ([i.d for i in sssp])

['s', 'y', 'z', 't', 'x']
[None, 's', 'y', 'y', 't']
[0, 5, 7, 8, 9]
